# 20장. 영상

### 20.2.2 환경 준비
```bash
conda env create -f env_CH20.yml
conda activate book-transfer-learning
git clone https://github.com/machrisaa/tensorflow-vgg tensorflow_vgg
```

### 20.2.5 데이터 훓어보기

In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm

import tarfile
dataset_folder_path = 'flower_photos'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile('flower_photos.tar.gz'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Flowers Dataset') as pbar:
        urlretrieve(
            'http://download.tensorflow.org/example_images/flower_photos.tgz',
            'flower_photos.tar.gz',
            pbar.hook)

if not isdir(dataset_folder_path):
    with tarfile.open('flower_photos.tar.gz') as tar:
        tar.extractall()
        tar.close()

Flowers Dataset: 229MB [00:06, 33.0MB/s]


In [2]:
import numpy as np
import csv
with open('labels.txt') as f:
    reader = csv.reader(f, delimiter='\n')
    labels = np.array([each for each in reader if len(each) > 0]).squeeze()

with open('codes.bin') as f:
    codes = np.fromfile(f, dtype=np.float32)
    codes = codes.reshape((len(labels), -1))

In [3]:
print(codes.shape)

(3670, 4096)


In [4]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(labels)

labels_vecs = lb.transform(labels)

print(labels[[0, 1000, 2000, 2500, 3000]])
print(labels_vecs[[0, 1000, 2000, 2500, 3000]])

['daisy' 'dandelion' 'roses' 'sunflowers' 'tulips']
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]


In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_idx, val_idx = next(ss.split(codes, labels_vecs))

half_val_len = int(len(val_idx)/2)
val_idx, test_idx = val_idx[:half_val_len], val_idx[half_val_len:]

train_x, train_y = codes[train_idx], labels_vecs[train_idx]
val_x, val_y = codes[val_idx], labels_vecs[val_idx]
test_x, test_y = codes[test_idx], labels_vecs[test_idx]

print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)
print("Test shapes (x, y):", test_x.shape, test_y.shape)

Train shapes (x, y): (2936, 4096) (2936, 5)
Validation shapes (x, y): (367, 4096) (367, 5)
Test shapes (x, y): (367, 4096) (367, 5)


### 20.2.6 모델 만들기

In [6]:
import tensorflow as tf

inputs_ = tf.placeholder(tf.float32, shape=[None, codes.shape[1]])
labels_ = tf.placeholder(tf.float32, shape=[None, labels_vecs.shape[1]])

fc = tf.layers.dense(inputs_, 256, activation=tf.nn.relu)
print(fc)
logits = tf.layers.dense(fc, labels_vecs.shape[1], activation=None)
print(logits)

Tensor("dense/Relu:0", shape=(?, 256), dtype=float32)
Tensor("dense_1/BiasAdd:0", shape=(?, 5), dtype=float32)


### 20.2.7 최적화 문제 설정

In [7]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels_, logits=logits)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer().minimize(cost)

predicted = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### 20.2.8 하이퍼 파라미터 설정

In [8]:
batch_size = 16
epochs = 10

def get_batches(x, y, batch_size=16):
    """ Return a generator that yields batches from arrays x and y. """
    n_batches = np.int32(np.floor(len(x) / batch_size))
    
    for ii in range(0, n_batches*batch_size, batch_size):
        # If we're not on the last batch, grab data with size batch_size
        if ii != (n_batches-1)*batch_size:
            X, Y = x[ii: ii+batch_size], y[ii: ii+batch_size] 
        # On the last batch, grab the rest of the data
        else:
            X, Y = x[ii:], y[ii:]
        yield X, Y

### 20.2.9 학습

In [9]:
!mkdir checkpoints

In [10]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        for x, y in get_batches(train_x, train_y, batch_size):
            feed = {inputs_: x,
                    labels_: y}
            loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
        feed = {inputs_: val_x,
                labels_: val_y}
        val_acc = sess.run(accuracy, feed_dict=feed)
        print("Epoch: {}/{}".format(e+1, epochs),
              "Validation Acc: {:.4f}".format(val_acc))
    saver.save(sess, "checkpoints/flowers.ckpt")

Epoch: 1/10 Validation Acc: 0.8283
Epoch: 2/10 Validation Acc: 0.8719
Epoch: 3/10 Validation Acc: 0.8529
Epoch: 4/10 Validation Acc: 0.8583
Epoch: 5/10 Validation Acc: 0.8529
Epoch: 6/10 Validation Acc: 0.8474
Epoch: 7/10 Validation Acc: 0.8665
Epoch: 8/10 Validation Acc: 0.8583
Epoch: 9/10 Validation Acc: 0.8610
Epoch: 10/10 Validation Acc: 0.8774


### 20.2.10 정확도

In [11]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    feed = {inputs_: test_x,
            labels_: test_y}
    test_acc = sess.run(accuracy, feed_dict=feed)
    print("Test accuracy: {:.4f}".format(test_acc))

INFO:tensorflow:Restoring parameters from checkpoints\flowers.ckpt
Test accuracy: 0.9101


## 20.3 Bottlne 특성 추출 방법

In [12]:
import os
data_dir = 'flower_photos/'
contents = os.listdir(data_dir)
classes = [each for each in contents if os.path.isdir(data_dir + each)]
classes

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [13]:
vgg_dir = 'tensorflow_vgg/'
if not isdir(vgg_dir):
    raise Exception("VGG directory doesn't exist!")

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(vgg_dir + "vgg16.npy"):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='VGG16 Parameters') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/content.udacity-data.com/nd101/vgg16.npy',
            vgg_dir + 'vgg16.npy',
            pbar.hook)
else:
    print("Parameter file already exists!")

VGG16 Parameters: 553MB [22:35, 408kB/s]


In [14]:
from tensorflow_vgg import vgg16, utils
# Set the batch size higher if you can fit in in your GPU memory
batch_size = 16
codes_list = []
labels = []
batch = []

codes = None

with tf.Session() as sess:
    
    vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    with tf.name_scope("content_vgg"):
        vgg.build(input_)

    for each in classes:
        print("Starting {} images".format(each))
        class_path = data_dir + each
        files = os.listdir(class_path)
        for ii, file in enumerate(files, 1):
            # Add images to the current batch
            # utils.load_image crops the input images for us, from the center
            img = utils.load_image(os.path.join(class_path, file))
            batch.append(img.reshape((1, 224, 224, 3)))
            labels.append(each)
            
            # Running the batch through the network to get the codes
            if ii % batch_size == 0 or ii == len(files):
                images = np.concatenate(batch)

                feed_dict = {input_: images}
                codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)# reshape
                
                # Here I'm building an array of the codes
                if codes is None:
                    codes = codes_batch
                else:
                    codes = np.concatenate((codes, codes_batch))
                
                # Reset to start building the next batch
                batch = []
                print('{} images processed'.format(ii))

e:\repos\tmp\dlopt\notebooks\tensorflow_vgg\vgg16.npy
npy file loaded
build model started
build model finished: 1s
Starting daisy images
C:\Users\HanByul\Anaconda3\envs\book-trasnfer-learning\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\HanByul\Anaconda3\envs\book-trasnfer-learning\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
16 images processed
32 images processed
48 images processed
64 images processed
80 images processed
96 images processed
112 images processed
128 images processed
144 images processed
160 images processed
176 images processed
192 images processed
208 images processed
224 images processed
240 ima

In [15]:
# write codes to file
with open('codes.bin', 'w') as f:
    codes.tofile(f)
    
# write labels to file
import csv
with open('labels.txt', 'w') as f:
    writer = csv.writer(f, delimiter='\n')
    writer.writerow(labels)